In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem.lancaster import LancasterStemmer
from nltk.probability import FreqDist
from collections import defaultdict
from heapq import nlargest
from collections import defaultdict
from bs4 import BeautifulSoup
import re

In [40]:
path = 'C:/Users/parth/Desktop/Git/Projects/IMDB/'
labelTrain = pd.read_csv(path+'labeledTrainData.tsv', header=0,delimiter="\t", quoting=3)
unlabelTrain = pd.read_csv(path+'unlabeledTrainData.tsv', header=0,delimiter="\t", quoting=3)
test = pd.read_csv(path+'testData.tsv', header=0,delimiter="\t", quoting=3)

In [41]:
labelTrain.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [42]:
test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [43]:

def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review,'lxml').get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))  


In [44]:
'''
def review_to_words(review):
    soup = BeautifulSoup(review,'lxml')
    text = soup.get_text()
    text = re.sub(r'\d+', '', text) # Removes all Number from the text
    text = text.replace('\\', "") # Removes “ from the text which cannot be removed by nltk
    words = word_tokenize(text.lower())
    customStopWords = set(stopwords.words('english')+list(punctuation) + ['``',"'s","'","''","'ve",'...'] )
    words = [word for word in words if word not in customStopWords]
    words = [LancasterStemmer().stem(word) for word in words]
    cleanReview =  " ".join(words)
    return cleanReview
'''

'\ndef review_to_words(review):\n    soup = BeautifulSoup(review,\'lxml\')\n    text = soup.get_text()\n    text = re.sub(r\'\\d+\', \'\', text) # Removes all Number from the text\n    text = text.replace(\'\\\', "") # Removes “ from the text which cannot be removed by nltk\n    words = word_tokenize(text.lower())\n    customStopWords = set(stopwords.words(\'english\')+list(punctuation) + [\'``\',"\'s","\'","\'\'","\'ve",\'...\'] )\n    words = [word for word in words if word not in customStopWords]\n    words = [LancasterStemmer().stem(word) for word in words]\n    cleanReview =  " ".join(words)\n    return cleanReview\n'

In [45]:
num_reviews = labelTrain["review"].size
print("Cleaning and parsing the training set movie reviews...\n")
clean_train_reviews = []
for i in range( 0, num_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%5000 == 0 ):
        print("Review %d of %d\n" % ( i+1, num_reviews ))                                                                    
    clean_train_reviews.append( review_to_words( labelTrain["review"][i] ))
    
print('Cleaning Done.')

Cleaning and parsing the training set movie reviews...

Review 5000 of 25000

Review 10000 of 25000

Review 15000 of 25000

Review 20000 of 25000

Review 25000 of 25000

Cleaning Done.


In [46]:
'''
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()
'''

'\nprint("Creating the bag of words...\n")\nfrom sklearn.feature_extraction.text import CountVectorizer\n\n# Initialize the "CountVectorizer" object, which is scikit-learn\'s\n# bag of words tool.  \nvectorizer = CountVectorizer(analyzer = "word",                                tokenizer = None,                                 preprocessor = None,                              stop_words = None,                                max_features = 5000) \n\n# fit_transform() does two functions: First, it fits the model\n# and learns the vocabulary; second, it transforms our training data\n# into feature vectors. The input to fit_transform should be a list of \n# strings.\ntrain_data_features = vectorizer.fit_transform(clean_train_reviews)\n\n# Numpy arrays are easy to work with, so convert the result to an \n# array\ntrain_data_features = train_data_features.toarray()\n'

In [47]:
print("Creating the bag of words tfidf...\n")
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Creating the bag of words tfidf...



In [48]:
train_data_features[0:2,0:10]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [49]:
vocab = vectorizer.get_feature_names()
print(vocab)

['abandoned', 'abc', 'abilities', 'ability', 'able', 'abraham', 'absence', 'absent', 'absolute', 'absolutely', 'absurd', 'abuse', 'abusive', 'abysmal', 'academy', 'accent', 'accents', 'accept', 'acceptable', 'accepted', 'access', 'accident', 'accidentally', 'accompanied', 'accomplished', 'according', 'account', 'accuracy', 'accurate', 'accused', 'achieve', 'achieved', 'achievement', 'acid', 'across', 'act', 'acted', 'acting', 'action', 'actions', 'activities', 'actor', 'actors', 'actress', 'actresses', 'acts', 'actual', 'actually', 'ad', 'adam', 'adams', 'adaptation', 'adaptations', 'adapted', 'add', 'added', 'adding', 'addition', 'adds', 'adequate', 'admire', 'admit', 'admittedly', 'adorable', 'adult', 'adults', 'advance', 'advanced', 'advantage', 'adventure', 'adventures', 'advertising', 'advice', 'advise', 'affair', 'affect', 'affected', 'afford', 'aforementioned', 'afraid', 'africa', 'african', 'afternoon', 'afterwards', 'age', 'aged', 'agent', 'agents', 'ages', 'aging', 'ago', 'ag

In [50]:
# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print(count, tag)

18.5347292772 abandoned
17.8203755752 abc
11.2588214944 abilities
39.3059610198 ability
93.8058200636 able
10.0245331268 abraham
12.9102059459 absence
9.59839591097 absent
36.7056696437 absolute
123.728137675 absolutely
31.7225101312 absurd
20.1353586463 abuse
9.94010788407 abusive
13.4072651097 abysmal
30.2793346104 academy
45.7770214395 accent
23.5023454552 accents
28.789028955 accept
15.7474434993 acceptable
15.5886409572 accepted
10.6432845703 access
31.0059040452 accident
19.7593099352 accidentally
9.66360235925 accompanied
14.9149033499 accomplished
26.7858719635 according
19.7033181044 account
10.4120870028 accuracy
31.0301891067 accurate
13.5825169923 accused
19.0705529852 achieve
13.9619319763 achieved
13.2661266666 achievement
10.7245079789 acid
74.5154152774 across
96.8527369381 act
67.416249144 acted
365.950947158 acting
217.313302918 action
27.6209430352 actions
9.02312129968 activities
161.804938339 actor
279.434262792 actors
96.9976571355 actress
40.2631639003 actresses


In [51]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, labelTrain["sentiment"] )

Training the random forest...


In [37]:
num_reviews = len(test["review"])
clean_test_reviews = [] 

print("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,num_reviews):
    if( (i+1) % 5000 == 0 ):
        print("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

print('Cleaning Done')

Cleaning and parsing the test set movie reviews...

Review 5000 of 25000

Review 10000 of 25000

Review 15000 of 25000

Review 20000 of 25000

Review 25000 of 25000

Cleaning Done


In [52]:
# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Sum up the counts of each vocabulary word
dist = np.sum(test_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print(count, tag)

0.733056396771 abandoned
15.8167531296 abc
0.938435306474 abilities
1.39892008117 ability
1.72500802878 able
5.97088489294 abraham
0.0956731747666 absence
0.809861901568 absent
0.227570696385 absolute
0.86958045446 absolutely
43.7476968822 absurd
0.797428542536 abuse
0.292978866167 abusive
0.323228812936 abysmal
35.991808057 academy
2.03404836868 accent
0.669367440286 accents
0.351987237275 accept
0.677001919723 acceptable
0.438565569424 accepted
18.226053307 access
2.64539954765 accident
0.303275263408 accidentally
0.0 accompanied
0.186615171075 accomplished
1.79440685472 according
41.1402424129 account
0.545378410154 accuracy
1.21498948712 accurate
0.075970812687 accused
0.643885836134 achieve
0.32251633611 achieved
0.907700238016 achievement
9.85807229701 acid
98.6751773996 across
2409.06949767 act
10.4744055743 acted
10.3744476824 acting
21.9140926344 action
1.50146839897 actions
0.885877831622 activities
8.35473589605 actor
10.0921273776 actors
144.992692272 actress
1.18896195562 

In [53]:
# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv(path+ "Bag_of_Words_model3.csv", index=False, quoting=3 )